In [1]:
import time

from utils import dataStructures, dataset, parallelizer
from utils.generators import markovian
import numpy as np

In [2]:
def print_correct(mela_id='22', janya_id='22_a', surr_serial=False, time_cal=None):
    dat = dataset.GetRagaSongCoordsConcat(mela_id)
    dat1 = dataset.GetRagaSongCoords(janya_id)
    if time_cal is not None:
        assert isinstance(time_cal, dict)
    print("Starting surrogate generation...")
    t1 = time.perf_counter()
    d_surr1 = markovian.GenerateForRaga(mela_id, 50, serial=surr_serial)
    d_surr2 = markovian.GenerateForRaga(janya_id, 50, serial=surr_serial)
    t2 = time.perf_counter()
    for key in dat:
        dat[key] = dataStructures.PackTuples(*dat[key])

    for key in dat1:
        dat1[key] = dataStructures.PackTuples(*dat1[key])

    d_fin1 = dataset.ConcatenateDicts(dat, d_surr1, dat1, d_surr2)
    adj = dataset.GetAdjustedMelodies(d_fin1)
    print("finding causality....")
    t3 = time.perf_counter()
    tc, totc, strengths = parallelizer.TrueLZPCausality_listParallel(list(adj.values()), list(adj.keys()), mela=dataset.GetRagaFromRagaId(mela_id))
    t4 = time.perf_counter()

    if time_cal is not None:
        time_cal["Surrogate{}".format(mela_id)] = t2 - t1
        time_cal["Causality{}".format(mela_id)] = t4 - t3
    return tc, totc, strengths

def get_generation_time50(mela_id='22', janya_id='22_a', surr_serial=True):
    dat = dataset.GetRagaSongCoordsConcat(mela_id)
    dat1 = dataset.GetRagaSongCoords(janya_id)
    print("Starting surrogate generation...")
    t1 = time.perf_counter()
    d_surr1 = markovian.GenerateForRaga(mela_id, 50, serial=surr_serial)
    d_surr2 = markovian.GenerateForRaga(janya_id, 50, serial=surr_serial)
    t2 = time.perf_counter()
    for key in dat:
        dat[key] = dataStructures.PackTuples(*dat[key])

    for key in dat1:
        dat1[key] = dataStructures.PackTuples(*dat1[key])

    d_fin1 = dataset.ConcatenateDicts(dat, d_surr1, dat1, d_surr2)
    adj = dataset.GetAdjustedMelodies(d_fin1)

    return t2 - t1


def ragaIterator(n_iters, mela, janya, write=True, times_it=None):
    if times_it is not None:
        assert isinstance(times_it, dict)
        times_it["Surrogate{}".format(mela)] = []
        times_it["Causality{}".format(mela)] = []
    tcs = []
    strengths = []
    totcs = []
    f = open("/home/abhisheknandekar/surr_stats/{}/truecauses50.txt".format(mela), "a")
    t1 = time.perf_counter()
    for i in range(n_iters):
        print("iteration: ", i + 1)
        tm = {}
        if i % 2 != 0:
            tc, totc, st = print_correct(mela, janya, surr_serial=True, time_cal=tm)
        else:
            tc, totc, st = print_correct(mela, janya, surr_serial=True, time_cal=tm)
        if times_it is not None:
            times_it["Surrogate{}".format(mela)].append(tm["Surrogate{}".format(mela)])
            times_it["Causality{}".format(mela)].append(tm["Causality{}".format(mela)])
        if write:
            f.write("iteration: {}/{}, {}/{}; time taken: {}/{}\n".format(i+1, n_iters, tc, totc,
                                                                          tm["Surrogate{}".format(mela)],
                                                                          tm["Causality{}".format(mela)]))
        t = time.perf_counter()
        print("time elapsed: ", (t - t1))
        t1 = t
        tcs.append(tc)
        totcs.append(totc)
        strengths.append(st)
    f.close()
    return tcs, totcs, strengths

avg_conns = lambda t_c : np.array(t_c).mean()

In [3]:
true_causes8, tot_causes8, super_strengths8 = ragaIterator(10, '8', '8_d')

iteration:  1
Starting surrogate generation...
finding causality....
6289 true causes out of 11448 total connections
time elapsed:  872.8921653660072
iteration:  2
Starting surrogate generation...
finding causality....
6049 true causes out of 11448 total connections
time elapsed:  903.6651761510002
iteration:  3
Starting surrogate generation...
finding causality....
5365 true causes out of 11448 total connections
time elapsed:  885.549076086987
iteration:  4
Starting surrogate generation...
finding causality....
6106 true causes out of 11448 total connections
time elapsed:  921.0132939720061
iteration:  5
Starting surrogate generation...
finding causality....
5421 true causes out of 11448 total connections
time elapsed:  861.0697943150008
iteration:  6
Starting surrogate generation...
finding causality....
5776 true causes out of 11448 total connections
time elapsed:  916.8995998679893
iteration:  7
Starting surrogate generation...
finding causality....
6447 true causes out of 11448 to

In [4]:
avg_conns(true_causes8)

6134.2

In [5]:
true_causes15, tot_causes15, super_strengths15 = ragaIterator(10, '15', '15_m')

iteration:  1
Starting surrogate generation...
finding causality....
9704 true causes out of 11024 total connections
time elapsed:  801.0766502699989
iteration:  2
Starting surrogate generation...
finding causality....
9229 true causes out of 11024 total connections
time elapsed:  856.9564508819894
iteration:  3
Starting surrogate generation...
finding causality....
9526 true causes out of 11024 total connections
time elapsed:  803.9674205709889
iteration:  4
Starting surrogate generation...
finding causality....
9610 true causes out of 11024 total connections
time elapsed:  835.3265625630156
iteration:  5
Starting surrogate generation...
finding causality....
9272 true causes out of 11024 total connections
time elapsed:  814.8349667309958
iteration:  6
Starting surrogate generation...
finding causality....
9363 true causes out of 11024 total connections
time elapsed:  866.0442233679933
iteration:  7
Starting surrogate generation...
finding causality....
9974 true causes out of 11024 t

In [6]:
avg_conns(true_causes15)

9550.0

In [7]:
true_causes22, tot_causes22, super_strengths22 = ragaIterator(10, '22', '22_a')

iteration:  1
Starting surrogate generation...
finding causality....
10355 true causes out of 10812 total connections
time elapsed:  1156.9201496790047
iteration:  2
Starting surrogate generation...
finding causality....
10352 true causes out of 10812 total connections
time elapsed:  1217.7449357580044
iteration:  3
Starting surrogate generation...
finding causality....
10279 true causes out of 10812 total connections
time elapsed:  1169.1817585229874
iteration:  4
Starting surrogate generation...
finding causality....
10296 true causes out of 10812 total connections
time elapsed:  1209.3371484230156
iteration:  5
Starting surrogate generation...
finding causality....
10317 true causes out of 10812 total connections
time elapsed:  1152.3510028019955
iteration:  6
Starting surrogate generation...
finding causality....
10445 true causes out of 10812 total connections
time elapsed:  1211.1644248839875
iteration:  7
Starting surrogate generation...
finding causality....
10430 true causes o

In [8]:
avg_conns(true_causes22)

10355.7

In [ ]:
true_causes28, tot_causes28, super_strengths28 = ragaIterator(10, '28', '28_k')

iteration:  1
Starting surrogate generation...
finding causality....
2768 true causes out of 10920 total connections
time elapsed:  1876.0696289879852
iteration:  2
Starting surrogate generation...
finding causality....
2860 true causes out of 10920 total connections
time elapsed:  1930.536380798003
iteration:  3
Starting surrogate generation...
finding causality....
2633 true causes out of 10920 total connections
time elapsed:  1845.8160294019908
iteration:  4
Starting surrogate generation...
finding causality....
3214 true causes out of 10920 total connections
time elapsed:  1914.550263779005
iteration:  5
Starting surrogate generation...
finding causality....
2932 true causes out of 10920 total connections
time elapsed:  1858.277088355011
iteration:  6
Starting surrogate generation...
finding causality....
2495 true causes out of 10920 total connections
time elapsed:  1931.7317277870025
iteration:  7
Starting surrogate generation...
finding causality....
3642 true causes out of 1092

In [ ]:
avg_conns(true_causes28)

In [3]:
true_causes29, tot_causes29, super_strengths29 = ragaIterator(10, '29', '29_h')


iteration:  1
Starting surrogate generation...
finding causality....
10420 true causes out of 11024 total connections
time elapsed:  1845.302905428
iteration:  2
Starting surrogate generation...
finding causality....
10419 true causes out of 11024 total connections
time elapsed:  1924.9272328999998
iteration:  3
Starting surrogate generation...
finding causality....
10420 true causes out of 11024 total connections
time elapsed:  1870.1058491670005
iteration:  4
Starting surrogate generation...
finding causality....
10423 true causes out of 11024 total connections
time elapsed:  1891.4265159909992
iteration:  5
Starting surrogate generation...
finding causality....
10421 true causes out of 11024 total connections
time elapsed:  1863.9266423650006
iteration:  6
Starting surrogate generation...
finding causality....
10424 true causes out of 11024 total connections
time elapsed:  1916.553394946999
iteration:  7
Starting surrogate generation...
finding causality....
10422 true causes out of

In [4]:
avg_conns(true_causes29)

10420.2

In [3]:
t_dict = {}

In [4]:
tc15, totc15, ss15 = ragaIterator(10, '15', '15_m', times_it=t_dict)

iteration:  1
Starting surrogate generation...
finding causality....
2581 true causes out of 3024 total connections
time elapsed:  243.97107663599996
iteration:  2
Starting surrogate generation...
finding causality....
2563 true causes out of 3024 total connections
time elapsed:  231.75201903700008
iteration:  3
Starting surrogate generation...
finding causality....
2546 true causes out of 3024 total connections
time elapsed:  244.29333018199986
iteration:  4
Starting surrogate generation...
finding causality....
2634 true causes out of 3024 total connections
time elapsed:  235.58424717700018
iteration:  5
Starting surrogate generation...
finding causality....
2718 true causes out of 3024 total connections
time elapsed:  238.30344848899995
iteration:  6
Starting surrogate generation...
finding causality....
2630 true causes out of 3024 total connections
time elapsed:  239.31799904600007
iteration:  7
Starting surrogate generation...
finding causality....
2500 true causes out of 3024 to

In [5]:
tc8, totc8, ss8 = ragaIterator(10, '8', '8_d', times_it=t_dict)

iteration:  1
Starting surrogate generation...
finding causality....
1700 true causes out of 3248 total connections
time elapsed:  266.77733208200016
iteration:  2
Starting surrogate generation...
finding causality....
1694 true causes out of 3248 total connections
time elapsed:  280.61473970999987
iteration:  3
Starting surrogate generation...
finding causality....
1677 true causes out of 3248 total connections
time elapsed:  265.4197135879999
iteration:  4
Starting surrogate generation...
finding causality....
1534 true causes out of 3248 total connections
time elapsed:  269.75789616300017
iteration:  5
Starting surrogate generation...
finding causality....
1923 true causes out of 3248 total connections
time elapsed:  274.9748076799997
iteration:  6
Starting surrogate generation...
finding causality....
1755 true causes out of 3248 total connections
time elapsed:  261.02548072000036
iteration:  7
Starting surrogate generation...
finding causality....
1593 true causes out of 3248 tota

In [6]:
tc22, totc22, ss22 = ragaIterator(10, '22', '22_a', times_it=t_dict)

iteration:  1
Starting surrogate generation...
finding causality....
2659 true causes out of 2912 total connections
time elapsed:  331.04775068599974
iteration:  2
Starting surrogate generation...
finding causality....
2680 true causes out of 2912 total connections
time elapsed:  333.4156561509999
iteration:  3
Starting surrogate generation...
finding causality....
2664 true causes out of 2912 total connections
time elapsed:  336.4771074
iteration:  4
Starting surrogate generation...
finding causality....
2699 true causes out of 2912 total connections
time elapsed:  351.9021121370006
iteration:  5
Starting surrogate generation...
finding causality....
2694 true causes out of 2912 total connections
time elapsed:  346.8454231279993
iteration:  6
Starting surrogate generation...
finding causality....
2658 true causes out of 2912 total connections
time elapsed:  336.54188785099996
iteration:  7
Starting surrogate generation...
finding causality....
2659 true causes out of 2912 total connec

In [7]:
tc28, totc28, ss28 = ragaIterator(10, '28', '28_k', times_it=t_dict)

iteration:  1
Starting surrogate generation...
finding causality....
772 true causes out of 2970 total connections
time elapsed:  533.7588575379996
iteration:  2
Starting surrogate generation...
finding causality....
1033 true causes out of 2970 total connections
time elapsed:  539.2980058150006
iteration:  3
Starting surrogate generation...
finding causality....
911 true causes out of 2970 total connections
time elapsed:  533.8941850080009
iteration:  4
Starting surrogate generation...
finding causality....
784 true causes out of 2970 total connections
time elapsed:  532.285945571999
iteration:  5
Starting surrogate generation...
finding causality....
907 true causes out of 2970 total connections
time elapsed:  524.6445312630003
iteration:  6
Starting surrogate generation...
finding causality....
939 true causes out of 2970 total connections
time elapsed:  524.527202366
iteration:  7
Starting surrogate generation...
finding causality....
923 true causes out of 2970 total connections
t

In [ ]:
tc29, totc29, ss29 = ragaIterator(10, '29', '29_h', times_it=t_dict)

iteration:  1
Starting surrogate generation...
finding causality....
2720 true causes out of 3024 total connections
time elapsed:  538.1137518249998
iteration:  2
Starting surrogate generation...
finding causality....
2720 true causes out of 3024 total connections
time elapsed:  528.3808838740006
iteration:  3
Starting surrogate generation...
finding causality....
2721 true causes out of 3024 total connections
time elapsed:  537.1447102499988
iteration:  4
Starting surrogate generation...
finding causality....
2721 true causes out of 3024 total connections
time elapsed:  532.4931929600007
iteration:  5
Starting surrogate generation...
finding causality....
2733 true causes out of 3024 total connections
time elapsed:  530.0610519940001
iteration:  6
Starting surrogate generation...
finding causality....
2720 true causes out of 3024 total connections
time elapsed:  540.2443348720008
iteration:  7
Starting surrogate generation...
finding causality....
2721 true causes out of 3024 total co

In [ ]:
t_dict